In [1]:
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
market = Market()
sec = SEC()

In [67]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [71]:
sp5

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [86]:
start_year = 2016
end_year = 2022
performance = []
sim = []
market.connect()
sec.connect()
for ticker in tqdm(sp5["Symbol"].unique()[64:]):
    try:
        prices = market.retrieve_ticker_prices("alpha_prices",ticker)
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        financials =sec.retrieve_filing_data(cik)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year - 1 for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        prices["close"] = [float(x) for x in prices["close"]]
        financials = financials.rename(columns={"filed":"date"})
        financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in financials["date"]]
        financials["year"] = [x.year for x in financials["date"]]
        financials["quarter"] = [x.quarter for x in financials["date"]]
        afc = financials
        relevant_col = []
        for col in afc:
            if len([x for x in afc[col].isna() if x == True]) / afc.index.size < 0.1:
                relevant_col.append(col)
        afc_final = afc[relevant_col]
        data = prices.merge(afc_final,on=["year","quarter"]).groupby(["year","quarter"]).mean()
        relevant_cols = [x for x in data.columns if x not in ['cik', 'sic', 'ein',
               'changed', 'wksi', 'fye', 'period', 'fy', 'prevrpt', 'detail', 'nciks']]
        data = data[relevant_cols].reset_index()
        data = data.fillna(method="bfill")
        factors = [x for x in data.columns if x not in ["close","year","quarter"]]
        for year in range(start_year,end_year):
            try:
                training_data = data[(data["year"]>=year - 6) & (data["year"]<year)].reset_index(drop=True)
                if training_data.index.size >= 24:
                    prediction_data = data[(data["year"]==year)]
                    refined_data = {"X":training_data[factors],"y":training_data[["close"]].rename(columns={"close":"y"})}
                    models = m.regression(refined_data)
                    models["year"] = year
                    for row in models.iterrows():
                        api = row[1]["api"]
                        prediction = row[1]["model"].predict(prediction_data[factors])
                        prediction_data[f"{api}_prediction"] = prediction
                        prediction_data[f"{api}_score"] = row[1]["score"]
                    included_columns = ["year","quarter","ticker"]
                    included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
                    prediction_data["ticker"] = ticker
                    prediction_data["year"] = prediction_data["year"] + 1
                    sim.append(prediction_data[included_columns])
            except Exception as e:
                print(year,str(e))
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

  0%|▌                                                                                                                                                                                                                                  | 1/441 [00:02<16:27,  2.24s/it]

'date'



  1%|██▌                                                                                                                                                                                                                              | 5/441 [00:51<1:37:42, 13.45s/it]

2021 Input contains NaN, infinity or a value too large for dtype('float64').



  2%|███▌                                                                                                                                                                                                                             | 7/441 [01:37<2:16:32, 18.88s/it]

...


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 438/441 [1:58:54<00:56, 18.78s/it]

2021 Found array with 0 sample(s) (shape=(0, 60)) while a minimum of 1 is required.



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 439/441 [1:59:14<00:38, 19.42s/it]

2021 Found array with 0 sample(s) (shape=(0, 51)) while a minimum of 1 is required.



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 441/441 [1:59:18<00:00, 16.23s/it]


In [74]:
market.connect()
prices = market.retrieve("alpha_prices")
market.disconnect()
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["close"] = [float(x) for x in prices["close"]]

In [75]:
simulation = prices.merge(pd.concat(sim),on=["year","quarter","ticker"],how="left").dropna()

In [76]:
simulation["average_prediction"] = [sum([row[1][col] for col in simulation.columns if "prediction" in col]) for row in simulation.iterrows()]

In [77]:
simulation["average_prediction"] = simulation["average_prediction"] / 3
simulation["delta"] = (simulation["average_prediction"] - simulation["close"]) / simulation["close"]

In [87]:
positions = 10
start_date = datetime(2016,1,1)
end_date = simulation["date"].max()
reqs = [1,3,5,10,15]
limit = 30
values = [True,False]
trades = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            date = start_date
            iterration_sim = simulation.copy()
            if value == False:
                iterration_sim["delta"] = iterration_sim["delta"] * -1
            while date < end_date:
                try:
                    todays_recs = iterration_sim[iterration_sim["date"]==date]
                    todays_recs.sort_values("delta",ascending=False,inplace=True)
                    if todays_recs.index.size > 0:
                        offering = todays_recs.iloc[position]
                        if offering["delta"] > req:
                            trade = offering
                            ticker = trade["ticker"]
                            buy_price = trade["close"]
                            exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:limit]
                            exits["gains"] = (exits["close"] - buy_price) / buy_price
                            exit = exits.sort_values("gains",ascending=False).iloc[0]
                            trade["sell_price"] = exit["close"]
                            trade["sell_date"] = exit["date"]
                            date = exit["date"] + timedelta(days=1)
                            trade["value"] = value
                            trade["req"] = req
                            trade["position"] = position
                            trades.append(trade)
                        else:
                            date = date + timedelta(days=1)
                    else:
                        date = date + timedelta(days=1)
                except Exception as e:
                    print(str(e))
                    date = date + timedelta(days=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:50<00:00, 85.47s/it]


In [88]:
t = pd.DataFrame(trades)

In [89]:
t["delta"] =  (t["sell_price"] - t["close"]) / t["close"]

In [90]:
analysis = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            position_trades = t[(t["value"]==value) & (t["req"]==req) & (t["position"]==position)].sort_values("date")
            position_trades["delta"] = (position_trades["sell_price"] - position_trades["close"]) / position_trades["close"]
            initial = 100 / positions
            for delta in position_trades["delta"]:
                initial = initial * (1+delta)
            analysis.append({
                "value":value,
                "req":req,
                "position":position,
                "pv":initial
            })
            

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 102.02it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 20.00it/s]


In [91]:
complete = pd.DataFrame(analysis).pivot_table(index=["value","req"],columns="position",values="pv").reset_index()

In [92]:
complete["pv"] = [sum([row[1][i] for i in range(1)]) for row in complete.iterrows()]

In [94]:
complete.sort_values("pv",ascending=False)

position,value,req,0,1,2,3,4,5,6,7,8,9,pv
5,True,1,397.804851,202.601194,72.968879,16.185437,17.565145,9.441056,11.760427,12.635732,13.051731,11.383977,397.804851
6,True,3,109.894199,23.297950,16.011249,16.120284,9.992508,9.992508,9.992508,10.000000,10.000000,10.000000,109.894199
0,False,1,84.610295,26.368185,14.011345,16.191182,15.448164,14.779685,14.407729,14.126803,14.321376,12.750812,84.610295
7,True,5,25.858493,11.908530,11.843069,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,25.858493
8,True,10,21.306531,11.908530,11.843069,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,21.306531
9,True,15,14.456474,11.908530,11.843069,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,14.456474
1,False,3,12.832338,11.491524,11.491524,11.491524,11.491524,11.491524,10.972431,10.267464,10.000000,10.000000,12.832338
2,False,5,12.832338,11.491524,11.491524,11.491524,11.491524,11.491524,10.972431,10.267464,10.000000,10.000000,12.832338
3,False,10,12.832338,11.491524,11.491524,11.491524,11.491524,11.491524,10.972431,10.267464,10.000000,10.000000,12.832338
4,False,15,12.252679,10.972431,10.972431,10.972431,10.972431,10.972431,10.972431,10.267464,10.000000,10.000000,12.252679


In [96]:
simulation

,date,open,high,low,close,volume,ticker,year,quarter,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,average_prediction,delta
4320,2017-01-03,178.8300,180.0000,177.2200,178.05,2510055,MMM,2017,1,227.302515,0.225100,173.031662,0.903745,149.597041,0.735451,183.310406,0.029545
4321,2017-01-04,178.0300,178.9000,177.6100,178.32,1541985,MMM,2017,1,227.302515,0.225100,173.031662,0.903745,149.597041,0.735451,183.310406,0.027986
4322,2017-01-05,178.2600,179.1400,176.8900,177.71,1447568,MMM,2017,1,227.302515,0.225100,173.031662,0.903745,149.597041,0.735451,183.310406,0.031514
4323,2017-01-06,177.2900,178.6000,175.8000,178.23,1625049,MMM,2017,1,227.302515,0.225100,173.031662,0.903745,149.597041,0.735451,183.310406,0.028505
4324,2017-01-09,178.3700,178.3800,177.2000,177.27,1622625,MMM,2017,1,227.302515,0.225100,173.031662,0.903745,149.597041,0.735451,183.310406,0.034075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2831903,2022-04-05,257.7800,266.6000,257.7500,262.94,1080968,BDX,2022,2,208.173035,0.334798,249.602325,0.870021,239.724891,-0.690316,232.500084,-0.115768
2831904,2022-04-06,261.9400,270.9700,261.1500,270.65,1593534,BDX,2022,2,208.173035,0.334798,249.602325,0.870021,239.724891,-0.690316,232.500084,-0.140957
2831905,2022-04-07,269.6500,274.4700,268.8300,273.52,1003331,BDX,2022,2,208.173035,0.334798,249.602325,0.870021,239.724891,-0.690316,232.500084,-0.149970
2831906,2022-04-08,274.0700,277.2900,274.0100,275.42,1397616,BDX,2022,2,208.173035,0.334798,249.602325,0.870021,239.724891,-0.690316,232.500084,-0.155834
